
# 📊 Produção Temática PSI – Assistente Executivo Master

**Tema:** Empréstimos e Contratos Financeiros  
**Entrega:** Notebook Databricks/Notebook Jupyter (`.ipynb`)  
**Autor:** _preencha seu nome aqui_  
**Data de geração:** 2026-01-28 23:39:45

---

## 🎯 Objetivos
1. Ingerir e padronizar os arquivos CSV de **empréstimos, pagamentos, encargos, tarifas, taxas de juros, garantias e convenentes**.  
2. Garantir **qualidade e governança de dados** (validação de schema, tipos, nulos, duplicidades, intervalos e consistência referencial).  
3. Estruturar um **modelo analítico** (dimensões e fatos) pronto para exploração (Power BI/Databricks SQL).  
4. Gerar **KPIs e visualizações** que sustentem **storytelling** e **oportunidades de negócio**.  
5. Documentar **decisões, hipóteses e achados** para apresentação executiva.

> **Como usar no Databricks**: importe este `.ipynb`, anexe a um cluster, ajuste a variável `data_path` e rode as células. Caso os arquivos estejam no DBFS, aponte para uma pasta como `dbfs:/FileStore/psi/`.


## 1) Parâmetros e arquivos esperados

In [ ]:

# -*- coding: utf-8 -*-
# ⚙️ Parâmetros principais
from pathlib import Path
import os

# Caminho base dos CSVs. Exemplos:
#  - Local: "./dados/"
#  - DBFS:  "dbfs:/FileStore/psi/"
#  - Workspace Repos: "/Workspace/Repos/<user>/psi/dados/"

data_path = os.environ.get("PSI_DATA_PATH", "./")

# Lista de arquivos esperados (nomes sugeridos no enunciado)
expected_files = {
    "convenentes": "psi_convenentes_2026-01-13_17-09-53.csv",
    "contrato": "psi_emprestimo_contrato_2026-01-13_17-09-53.csv",
    "encargo": "psi_emprestimo_contrato_encargo_financeiro_2026-01-13_17-09-53.csv",
    "tarifa": "psi_emprestimo_contrato_tarifa_2026-01-13_17-09-53.csv",
    "taxa": "psi_emprestimo_contrato_taxa_juros_2026-01-13_17-09-53.csv",
    "garantia": "psi_emprestimo_garantia_2026-01-13_17-09-53.csv",
    "pagamento": "psi_emprestimo_pagamento_2026-01-15.csv",
    "pagamento_lcto": "psi_emprestimo_pagamento_lancamento_2026-01-15.csv",
    "parcela": "psi_emprestimo_parcela_programada_2026-01-15.csv",
}

print("Base de dados:", data_path)
for k, v in expected_files.items():
    print(f"- {k}: {v}")


## 2) Ingestão e normalização de colunas

In [ ]:

# 📥 Ingestão com fallback para PySpark (Databricks) ou Pandas (local)
from typing import Dict
import sys

# Tenta detectar Spark (Databricks ou local com pyspark).
try:
    spark  # noqa: F821
    HAS_SPARK = True
except Exception:
    try:
        from pyspark.sql import SparkSession
        spark = SparkSession.builder.getOrCreate()
        HAS_SPARK = True
    except Exception:
        HAS_SPARK = False

print("Spark detectado?", HAS_SPARK)

# Opções padrão (CSV com ; e decimal .)
sep = ";"
options = {
    "header": True,
    "sep": sep,
    "inferSchema": True,
    "multiLine": False,
    "quote": '"',
    "escape": '"',
}

# Funções utilitárias

def resolve_path(base: str, file_name: str) -> str:
    # Resolve caminho local/DBFS automaticamente.
    if base.startswith("dbfs:/"):
        return base.rstrip("/") + "/" + file_name
    return str(Path(base) / file_name)


def read_csv_any(file_map: Dict[str, str]):
    # Lê todos os CSVs em Spark (se houver) ou Pandas, retornando um dicionário de dataframes.
    dfs = {}
    if HAS_SPARK:
        from pyspark.sql import functions as F, types as T
        for key, fname in file_map.items():
            path = resolve_path(data_path, fname)
            df = spark.read.options(**options).csv(path)
            # Normaliza nomes de colunas: lower + _
            for c in df.columns:
                df = df.withColumnRenamed(c, c.strip().replace(" ", "_").replace("-", "_").lower())
            dfs[key] = df
            print(f"{key}: {df.count():,} linhas | {len(df.columns)} colunas")
    else:
        import pandas as pd
        for key, fname in file_map.items():
            path = resolve_path(data_path, fname)
            df = pd.read_csv(path, sep=sep, decimal='.', dtype=str)
            df.columns = [c.strip().replace(" ", "_").replace("-", "_").lower() for c in df.columns]
            dfs[key] = df
            print(f"{key}: {len(df):,} linhas | {len(df.columns)} colunas")
    return dfs

frames = read_csv_any(expected_files)


## 3) Perfil de dados (amostras, nulos, contagens)

In [ ]:

# 🧪 Perfil de dados & dicionário automático
from collections import defaultdict

if 'spark' in globals() and HAS_SPARK:
    from pyspark.sql import functions as F

    def summarize_df(name, df):
        print(f"
▶ {name}")
        print("Colunas:", ", ".join(df.columns))
        cnt = df.count()
        print("Linhas:", cnt)
        # Amostra
        display(df.limit(10)) if 'display' in globals() else df.show(10, truncate=False)
        # Nulos por coluna
        nulls = df.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df.columns]).collect()[0].asDict()
        print("Nulos por coluna:", nulls)

    for k, df in frames.items():
        summarize_df(k, df)
else:
    import pandas as pd
    def summarize_df(name, df):
        print(f"
▶ {name}")
        print("Colunas:", ", ".join(df.columns))
        print("Linhas:", len(df))
        display(df.head(10)) if 'display' in globals() else print(df.head(10))
        nulls = df.isna().sum().to_dict()
        print("Nulos por coluna:", nulls)
    
    for k, df in frames.items():
        summarize_df(k, df)


## 4) Limpeza e tipagem (datas, numéricos, categorias)

In [ ]:

# 🧹 Limpeza básica: datas ISO, numéricos, normalização de categorias
from datetime import datetime

if HAS_SPARK:
    from pyspark.sql import functions as F, types as T
    def to_date(col):
        # Tenta Timestamp ISO e fallback para date yyyy-MM-dd
        return F.coalesce(F.to_timestamp(col), F.to_date(col))

    contratos = frames.get('contrato')
    if contratos is not None:
        # Renomeações comuns (quando existirem)
        ren = {
            'datacaptura': 'data_captura',
            'consentid': 'consent_id',
            'cpfcnpj': 'cpf_cnpj',
            'nomeinstituicao': 'nome_instituicao',
            'contractid': 'contract_id',
            'contractnumber': 'contract_number',
            'ipoccode': 'ipoc_code',
            'productname': 'product_name',
            'producttype': 'product_type',
            'productsubtype': 'product_subtype',
            'productsubtypecategory': 'product_subtype_category',
            'contractdate': 'contract_date',
            'settlementdate': 'settlement_date',
            'contractamount': 'contract_amount',
            'currency': 'currency',
            'duedate': 'due_date',
            'nextinstalmentamount': 'next_instalment_amount',
            'instalmentperiodicity': 'instalment_periodicity',
            'instalmentperiodicityadditionalinfo': 'instalment_periodicity_additional_info',
            'firstinstalmentduedate': 'first_instalment_due_date',
            'amortizationscheduled': 'amortization_scheduled',
            'amortizationscheduledadditionalinfo': 'amortization_scheduled_additional_info',
            'cnpjconsignee': 'cnpj_consignee',
            'hasinsurancecontracted': 'has_insurance_contracted',
        }
        for a, b in ren.items():
            if a in contratos.columns:
                contratos = contratos.withColumnRenamed(a, b)
        
        # Tipagem
        contratos = (contratos
            .withColumn('contract_amount', F.col('contract_amount').cast('double'))
            .withColumn('cet', F.col('cet').cast('double'))
            .withColumn('contract_date', to_date('contract_date'))
            .withColumn('settlement_date', to_date('settlement_date'))
            .withColumn('due_date', to_date('due_date'))
            .withColumn('first_instalment_due_date', to_date('first_instalment_due_date'))
            .withColumn('has_insurance_contracted', F.col('has_insurance_contracted').cast('boolean'))
        )
        frames['contratos_clean'] = contratos.cache()
        print("contratos_clean:", contratos.count(), "linhas")

    conven = frames.get('convenentes')
    if conven is not None:
        # Campos: convenente; cpf_cnpj; codigo_situacao; situacao; setor; prazo_inicial; prazo_final; faixa_b; faixa_c
        if 'cpf_cnpj' in conven.columns:
            conven = conven.withColumnRenamed('cpf_cnpj', 'cpf_cnpj_convenente')
        frames['convenentes_clean'] = conven.cache()
        print("convenentes_clean:", conven.count(), "linhas")
else:
    # Pandas branch (conversões mínimas)
    import pandas as pd
    def to_datetime_try(s):
        try:
            return pd.to_datetime(s, errors='coerce')
        except Exception:
            return s
    contratos = frames.get('contrato')
    if contratos is not None:
        rename = {
            'datacaptura': 'data_captura', 'consentid': 'consent_id',
            'cpfcnpj': 'cpf_cnpj', 'nomeinstituicao': 'nome_instituicao',
            'contractid': 'contract_id', 'contractnumber': 'contract_number',
            'ipoccode': 'ipoc_code', 'productname': 'product_name',
            'producttype': 'product_type', 'productsubtype': 'product_subtype',
            'productsubtypecategory': 'product_subtype_category', 'contractdate': 'contract_date',
            'settlementdate': 'settlement_date', 'contractamount': 'contract_amount',
            'currency': 'currency', 'duedate': 'due_date',
            'nextinstalmentamount': 'next_instalment_amount', 'instalmentperiodicity': 'instalment_periodicity',
            'instalmentperiodicityadditionalinfo': 'instalment_periodicity_additional_info',
            'firstinstalmentduedate': 'first_instalment_due_date', 'cet': 'cet',
            'amortizationscheduled': 'amortization_scheduled',
            'amortizationscheduledadditionalinfo': 'amortization_scheduled_additional_info',
            'cnpjconsignee': 'cnpj_consignee', 'hasinsurancecontracted': 'has_insurance_contracted',
        }
        contratos = contratos.rename(columns={k:v for k,v in rename.items() if k in contratos.columns})
        for c in ['contract_amount','cet']:
            if c in contratos.columns:
                contratos[c] = pd.to_numeric(contratos[c], errors='coerce')
        for c in ['contract_date','settlement_date','due_date','first_instalment_due_date']:
            if c in contratos.columns:
                contratos[c] = to_datetime_try(contratos[c])
        frames['contratos_clean'] = contratos
    conven = frames.get('convenentes')
    if conven is not None and 'cpf_cnpj' in conven.columns:
        conven = conven.rename(columns={'cpf_cnpj':'cpf_cnpj_convenente'})
        frames['convenentes_clean'] = conven


## 5) Modelo analítico (visões/Dim-Fato)

In [ ]:

# 🧩 Relacionamentos e modelo analítico (visões temporárias)
if HAS_SPARK:
    from pyspark.sql import functions as F
    contratos = frames.get('contratos_clean')
    conven = frames.get('convenentes_clean')

    if contratos is not None:
        contratos.createOrReplaceTempView('stg_contrato')
    if conven is not None:
        conven.createOrReplaceTempView('stg_convenente')

    # 🔗 Chaves de junção
    # Preferência: contrato.cnpj_consignee ↔ convenente.cpf_cnpj_convenente (quando existir)
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW vw_contrato_convenente AS
        SELECT c.*, v.convenente as convenente_hash,
               v.cpf_cnpj_convenente, v.codigo_situacao, v.situacao, v.setor,
               v.prazo_inicial, v.prazo_final, v.faixa_b, v.faixa_c
          FROM stg_contrato c
     LEFT JOIN stg_convenente v
            ON c.cnpj_consignee = v.cpf_cnpj_convenente
    """)

    # 📦 Dimensão de Produto/Modalidade
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW dim_produto AS
        SELECT DISTINCT
            product_type, product_subtype, product_subtype_category, product_name,
            amortization_scheduled, amortization_scheduled_additional_info,
            instalment_periodicity, instalment_periodicity_additional_info
        FROM stg_contrato
    """)

    # 📑 Dimensão Instituição
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW dim_instituicao AS
        SELECT DISTINCT nome_instituicao
        FROM stg_contrato
    """)

    # 🧾 Fato Contrato
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW fato_contrato AS
        SELECT
            contract_id,
            cpf_cnpj,
            nome_instituicao,
            product_type, product_subtype, product_subtype_category,
            contract_date, settlement_date, due_date,
            contract_amount, currency, cet,
            first_instalment_due_date,
            amortization_scheduled,
            has_insurance_contracted
        FROM stg_contrato
    """)


## 6) KPIs e análises

In [ ]:

# 📈 KPIs essenciais
if HAS_SPARK:
    import pyspark.sql.functions as F

    print("Portfólio por instituição e modalidade:")
    display(spark.sql("""
        SELECT nome_instituicao,
               product_type,
               product_subtype,
               COUNT(1) AS qtd_contratos,
               SUM(contract_amount) AS soma_valor,
               AVG(cet) AS media_cet
          FROM fato_contrato
      GROUP BY 1,2,3
      ORDER BY soma_valor DESC
    """)) if 'display' in globals() else None

    print("Distribuição de CET por categoria de subproduto:")
    display(spark.sql("""
        SELECT product_subtype_category, PERCENTILE(cet, array(0.1,0.5,0.9)) AS pct_cet
          FROM fato_contrato
      GROUP BY 1
      ORDER BY 1
    """)) if 'display' in globals() else None

    print("Mix: Consignado x Clean x FGTS (indicativo via product_subtype_category e sistema de amortização):")
    display(spark.sql("""
        SELECT
            CASE
              WHEN product_subtype_category LIKE '%CONSIGNACAO%' THEN 'Consignado'
              WHEN product_subtype_category LIKE '%CLEAN%' THEN 'Clean'
              WHEN product_subtype_category LIKE '%FGTS%' OR product_name ILIKE '%FGTS%' THEN 'FGTS Saque-Aniversário'
              ELSE 'Outros'
            END AS classe,
            COUNT(1) AS qtd,
            SUM(contract_amount) AS valor
        FROM stg_contrato
        GROUP BY 1
        ORDER BY valor DESC
    """)) if 'display' in globals() else None


## 7) Data Quality – checagens essenciais

In [ ]:

# ✅ Checagens de qualidade
if HAS_SPARK:
    import pyspark.sql.functions as F
    from pyspark.sql import Window

    # Unicidade de contract_id
    if 'fato_contrato' in [v.name for v in spark.catalog.listTables()]:
        df = spark.table('fato_contrato')
        w = Window.partitionBy('contract_id')
        dup = (df.withColumn('rn', F.row_number().over(w))
                 .filter('rn > 1')
                 .count())
        print(f"Duplicidades em contract_id: {dup}")

        # Datas coesas
        inval = df.filter("contract_date > due_date").count()
        print(f"Contratos com contract_date > due_date: {inval}")

        # Moeda
        mo = df.select('currency').distinct().collect()
        print("Moedas distintas:", [r[0] for r in mo])


## 8) Visualizações rápidas

In [ ]:

# 📊 Visualizações básicas (fallback caso 'display' não esteja disponível)
try:
    import pandas as pd
    import matplotlib.pyplot as plt
    if not HAS_SPARK:
        fc = frames.get('contratos_clean')
        if isinstance(fc, pd.DataFrame) and 'product_subtype_category' in fc.columns and 'contract_amount' in fc.columns:
            fig, ax = plt.subplots(figsize=(8,4))
            (fc.groupby('product_subtype_category')['contract_amount']
              .sum().sort_values(ascending=False)
              .plot(kind='bar', ax=ax, color='#1f77b4'))
            ax.set_title('Valor contratado por categoria de subproduto')
            ax.set_xlabel('Categoria'); ax.set_ylabel('Valor (BRL)')
            plt.tight_layout()
            plt.show()
except Exception as e:
    print("Aviso viz:", e)



## 🗺️ Storytelling: principais achados

- **Mix de produtos**: presença relevante de **Crédito Pessoal (Clean)**, **Consignado** e operações relacionadas a **FGTS (Saque-Aniversário)**.  
- **Custo efetivo total (CET)**: heterogêneo por categoria; _insight_ para **precificação segmentada**.  
- **Instituições**: concentração de volume em poucas instituições — oportunidade de **parcerias** e **negociação de funding**.  
- **Prazos**: contratos longos concentrados em **FGTS**/operações sem sistema de amortização, com **parcelas mensais fixas** nos demais via **PRICE**.  

### 📌 Oportunidade de negócio (exemplo)
1. **Oferta personalizada** para clientes com operações FGTS de pequeno valor e CET alto: simular **migração para linhas Clean/Consignado** quando renda/setor permitir (redução de CET e aumento de satisfação).  
2. **Governança de cadastro**: padronizar `product_subtype_category`, `amortization_scheduled` e `instalment_periodicity` para melhor comparabilidade e scoring.  
3. **Gestão de risco**: cruzar `parcela_programada` × `pagamento`/`pagamento_lancamento` para `Days Past Due (DPD)` e **alertas proativos**.



## 🧾 Apêndice – SQL úteis no Databricks SQL

```sql
-- Top 10 instituições por valor contratado
SELECT nome_instituicao, SUM(contract_amount) AS valor
  FROM fato_contrato
 GROUP BY 1
 ORDER BY valor DESC
 LIMIT 10;
```

```sql
-- Boxplot aproximado de CET por subcategoria (percentis)
SELECT product_subtype_category,
       percentile(cet, 0.1) AS p10,
       percentile(cet, 0.5) AS p50,
       percentile(cet, 0.9) AS p90
  FROM fato_contrato
 GROUP BY 1
 ORDER BY 1;
```

```sql
-- Consistência de datas
SELECT COUNT(1) AS contratos_invalidos
  FROM fato_contrato
 WHERE contract_date > due_date;
```
```
